In [2]:
import mne
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.metrics import Precision, Recall

In [22]:
import os

print(os.listdir("./data/"))
filepath = './data/p002_MI_withbins.set'

['p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be.fdt', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be.set', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be_ar.fdt', 'p002_MI_markers_elist_f_cf pruned with ICA_bins_impel_be_ar.set', 'p002_MI_withbins.set', 'test01.txt', 'test04.txt']


In [135]:
epochs = mne.read_epochs_eeglab(filepath)
df = epochs.to_data_frame()

Extracting parameters from D:\Development\EEG ANALITICS\data\p002_MI_withbins.set...
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Ready.


C:\Users\Kiselev\AppData\Local\Temp\1\ipykernel_19652\2379057301.py:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.read_epochs_eeglab(filepath)


In [136]:
from scipy.io import loadmat

def mat_struct_to_dict(mat_struct_obj):
    return {field: getattr(mat_struct_obj, field) for field in mat_struct_obj._fieldnames}
eeglab_data = loadmat(filepath, struct_as_record=False, squeeze_me=True)

events_data = eeglab_data['event']
df['codelabel'] = pd.NA
for i, event in enumerate(events_data):
    label = mat_struct_to_dict(event)['codelabel']
    df.loc[i, "codelabel"] = label

In [137]:
df = df.dropna()
df["codelabel"]

0       s0
1       s0
2      stf
3       s0
4       s0
      ... 
457     s0
458    stl
459     s0
460     s0
461     s0
Name: codelabel, Length: 462, dtype: object

In [138]:
# двигает рукой
stf = []
# частично двигает рукой
sts = []
# не двигает рукой - но представляет
stl = []
# не двигает вообще
s0 = []
for index, row in df.iterrows():
  if 'stf' in row['codelabel']:
   stf.append(index)
  elif 'sts' in row['codelabel']:
    sts.append(index)
  elif 'stl' in row['codelabel']:
    stl.append(index)
  elif 's0' in row['codelabel']:
    s0.append(index)

In [139]:
df2 = df.drop('codelabel', axis=1)
df2['codelabel'] = pd.NA
for el in s0:
  df2.at[el, 'codelabel'] = 0
for el in stf:
  df2.at[el, 'codelabel'] = 1
for el in sts:
  df2.at[el, 'codelabel'] = 2
for el in stl:
  df2.at[el, 'codelabel'] = 3

In [140]:
df2

,time,condition,epoch,F7,F3,Fz,FC5,FC1,T7,C3,...,FC3,FCz,C5,C1,CP3,P5,P1,FT7,TP7,codelabel
0,-3.000000,0/0/B1(111),0,-28.969856,-4.137545,-0.255840,-9.641929,-2.538495,-6.198280,-2.303777,...,-2.545691,-0.849673,-1.208414,-1.937044,0.538882,2.164967,1.455608,-0.315933,-8.471616,0
1,-2.999023,0/0/B1(111),0,-28.903843,-4.122650,-0.257706,-9.633248,-2.541231,-6.191755,-2.311632,...,-2.539751,-0.852515,-1.234916,-1.939596,0.529298,2.160731,1.463335,-0.232176,-8.480064,0
2,-2.998047,0/0/B1(111),0,-28.825142,-4.105039,-0.260104,-9.622827,-2.544465,-6.183642,-2.320876,...,-2.532714,-0.855877,-1.266197,-1.942527,0.517940,2.155664,1.472435,-0.132545,-8.489736,1
3,-2.997070,0/0/B1(111),0,-28.750874,-4.088592,-0.262575,-9.612904,-2.547488,-6.175588,-2.329464,...,-2.526122,-0.859026,-1.295355,-1.945159,0.507298,2.150860,1.480893,-0.038761,-8.498386,0
4,-2.996094,0/0/B1(111),0,-28.671953,-4.071247,-0.265370,-9.602285,-2.550681,-6.166713,-2.338484,...,-2.519155,-0.862355,-1.326064,-1.947852,0.496047,2.145736,1.489783,0.060720,-8.507207,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,-2.553711,0/0/B1(111),0,8.679171,5.863000,2.315423,-0.502896,2.090754,-3.036660,3.410460,...,3.730740,2.696402,-1.366479,1.574120,2.009455,-0.678801,0.180952,-5.784262,-2.928797,0
458,-2.552734,0/0/B1(111),0,8.734139,5.897024,2.340048,-0.472778,2.100465,-3.021600,3.425903,...,3.744581,2.699100,-1.357450,1.569355,2.023538,-0.671098,0.175322,-5.822018,-2.918742,3
459,-2.551758,0/0/B1(111),0,8.789356,5.930980,2.364521,-0.442625,2.109986,-3.006087,3.441116,...,3.758270,2.701607,-1.348618,1.564448,2.037477,-0.663358,0.169696,-5.858509,-2.908505,0
460,-2.550781,0/0/B1(111),0,8.844829,5.964865,2.388838,-0.412435,2.119316,-2.990114,3.456097,...,3.771809,2.703922,-1.339978,1.559397,2.051273,-0.655587,0.164073,-5.893715,-2.898086,0


In [141]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df2.drop(columns=['epoch','time','condition', 'codelabel'])
y = df2['codelabel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [147]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

# Создание нейронной сети
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение модели
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3512 - loss: 1.4540 - val_accuracy: 0.8417 - val_loss: 0.8548
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7985 - loss: 0.8062 - val_accuracy: 0.8417 - val_loss: 0.5949
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7857 - loss: 0.6553 - val_accuracy: 0.8417 - val_loss: 0.5345
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7717 - loss: 0.6479 - val_accuracy: 0.8417 - val_loss: 0.5176
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7720 - loss: 0.6297 - val_accuracy: 0.8417 - val_loss: 0.5010
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7832 - loss: 0.5894 - val_accuracy: 0.8417 - val_loss: 0.4975
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7897 - loss: 0.5787 - val_accuracy: 0.8417 - val_loss: 0.4964
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7710 - loss: 0.5950 - val_accuracy: 0.8417 - val_loss

In [150]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model_tree = SVC(kernel='linear') 
model_tree.fit(X_train, y_train)
y_pred = model_tree.predict(X_test)
# Оценка модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Дополнительные метрики
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.84
Confusion Matrix:
[[117   0   0   0]
 [  8   0   0   0]
 [  6   0   0   0]
 [  8   0   0   0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       117
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         8

    accuracy                           0.84       139
   macro avg       0.21      0.25      0.23       139
weighted avg       0.71      0.84      0.77       139



D:\Development\EEG ANALITICS\env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Development\EEG ANALITICS\env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Development\EEG ANALITICS\env\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [120]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8483 - loss: 0.5295
Loss: 0.5160
Accuracy: 0.8417


In [121]:
model.save('model.keras')

In [131]:
def get_act_name(code):
    if code  == 0:
        return "s0"
    elif code == 1:
        return "stf"
    elif code == 2:
        return "sts"
    elif code == 3:
        return "stl"

In [151]:
test_num = 19
predicted = get_act_name(np.argmax(model.predict(X_test[test_num].reshape(1, 24))))
correct = get_act_name(list(y_test)[test_num])
print(f"predicted num is `{predicted}` and correct num is `{correct}`")

predicted num is `s0` and correct num is `stl`


In [113]:
# # Загрузка модели
# model = tf.keras.models.load_model('model.keras')

# Инициализация конвертера
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Конвертация в формат TFLite
tflite_model = converter.convert()

# Сохранение TFLite модели в файл
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Kiselev\AppData\Local\Temp\1\tmpxcey7ze0\assets


INFO:tensorflow:Assets written to: C:\Users\Kiselev\AppData\Local\Temp\1\tmpxcey7ze0\assets


Saved artifact at 'C:\Users\Kiselev\AppData\Local\Temp\1\tmpxcey7ze0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 24), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  2833068680112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833050998560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833553463600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833553464832: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [114]:
file_size = os.path.getsize("./model.tflite")

# Вывод размера файла
print(f"Размер файла: {file_size} байт")

Размер файла: 9152 байт


In [115]:
model_path= "model.tflite"

interpreter = tf.lite.Interpreter(model_path=model_path)


interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
print(input_details[0]['shape'])
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

[ 1 24]


In [117]:
#input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = X_test[19]
print(f"data on input: {input_data}")
print(f"It has action: {get_act_name(list(y_test)[19])}")
input_reshaped = input_data.reshape(input_shape)

# входные данные
interpreter.set_tensor(input_details[0]['index'], input_reshaped)
print("Input details:", input_details)
#  предсказание
interpreter.invoke()

# выходные данные
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Predicted output:", output_data)
index_max = np.argmax(output_data)
print(index_max)

data on input: [ 1.2075263   1.7418103   1.8990794   1.6007686   1.8169459  -0.32188284
  1.8492963   1.7219074  -0.04727307  1.7055408  -0.87768924 -1.1796374
 -1.7522906   1.381032    1.945708    1.7226847   1.7566726   1.5064378
  1.3861762   1.878178   -0.8627352  -1.7587407  -1.716121    1.1047686 ]
It has action: stl
Input details: [{'name': 'serving_default_keras_tensor_12:0', 'index': 0, 'shape': array([ 1, 24]), 'shape_signature': array([-1, 24]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Predicted output: [[0.8406498  0.00273563 0.00723652 0.14937805]]
0


In [118]:
np.savetxt('X_test.csv', X_test, delimiter=',', fmt='%f', header='', comments='')